In [13]:
from bs4 import BeautifulSoup
import pandas as pd 
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time


In [10]:
url_unique ='https://www.superga.com'
url_base = 'https://www.superga.com/collections/'
url_page = 'womens-platform?filter.p.m.custom.seasons=Spring%2FSummer'
url = url_base+url_page
header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"}
print(url)
page = requests.get(url, headers=header)
print(page)


https://www.superga.com/collections/womens-platform?filter.p.m.custom.seasons=Spring%2FSummer
<Response [200]>


In [11]:

soup = BeautifulSoup(page.text)
stivaletti = soup.find_all('a', class_="grid-product__link")

print(stivaletti)


2790 PLATFORM<span class="colorname-title">WHITE</span>omen's Platform " href=" /products/lady-shoes-woman-2790-platform-white-s9111lw-901?_pos=1&amp;_fid=92de5ca23&amp;_ss=c ">
2790 PLATFORM<span class="colorname-title">FULL BLACK</span>omen's Platform " href=" /products/lady-shoes-woman-2790-platform-full-black-s9111lw-996?_pos=2&amp;_fid=92de5ca23&amp;_ss=c ">
2790 PLATFORM<span class="colorname-title">BLACK-FWHITE</span>en's Platform " href=" /products/lady-shoes-woman-2790-platform-black-fwhite-s9111lw-f83?_pos=3&amp;_fid=92de5ca23&amp;_ss=c ">
2790 PLATFORM<span class="colorname-title">NAVY-FWHITE</span>men's Platform " href=" /products/lady-shoes-woman-2790-platform-navy-fwhite-s9111lw-f43?_pos=4&amp;_fid=92de5ca23&amp;_ss=c ">
2790 PLATFORM<span class="colorname-title">BEIGE GESSO</span>men's Platform " href=" /products/lady-shoes-woman-2790-platform-beige-gesso-s9111lw-394?_pos=5&amp;_fid=92de5ca23&amp;_ss=c ">
2790 PLATFORM<span class="colorname-title">BLUE SPECTRUM-FAVORIO</

In [12]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode (no UI)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Load the page
url_stivaletti = stivaletti[0].get('href')  # Get the link from stivaletti
url1 = url_unique + url_stivaletti  # Construct the full URL
driver.get(url1)

# Wait for JavaScript to render the page (Adjust time based on page load)
time.sleep(5)

# Now that the page is rendered, extract the page source
html_content = driver.page_source

# Parse the page source with BeautifulSoup
soup_stivaletti = BeautifulSoup(html_content, 'html.parser')

print(soup_stivaletti)

WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=131.0.6778.109)
Stacktrace:
	GetHandleVerifier [0x00303433+25059]
	(No symbol) [0x0028CE34]
	(No symbol) [0x0016BEC3]
	(No symbol) [0x00169CF4]
	(No symbol) [0x0015D978]
	(No symbol) [0x0015F270]
	(No symbol) [0x0015DBC7]
	(No symbol) [0x0015D567]
	(No symbol) [0x0015D481]
	(No symbol) [0x0015B91B]
	(No symbol) [0x0015BE6B]
	(No symbol) [0x0016E8BE]
	(No symbol) [0x001EBF47]
	(No symbol) [0x001D1EFC]
	(No symbol) [0x001EB51E]
	(No symbol) [0x001D1C96]
	(No symbol) [0x001A3FAC]
	(No symbol) [0x001A4F3D]
	GetHandleVerifier [0x005F5593+3113795]
	GetHandleVerifier [0x0060A25A+3198986]
	GetHandleVerifier [0x00602A32+3168226]
	GetHandleVerifier [0x003A32A0+680016]
	(No symbol) [0x0029577D]
	(No symbol) [0x00292A28]
	(No symbol) [0x00292BC5]
	(No symbol) [0x00285820]
	BaseThreadInitThunk [0x76127BA9+25]
	RtlInitializeExceptionChain [0x7714C0CB+107]
	RtlClearBits [0x7714C04F+191]
	(No symbol) [0x0000

In [ ]:

rows=[]
nomi = []
colore = []
#prezzo_scontato = []
prezzo_originale = []
sizes = []

name_stivaletti = soup_stivaletti.find('h1',class_='h2 product-single__title')
colore_stivaletti = soup_stivaletti.find('span', class_='cb-italic cb-block cb-font-500')
#prezzo_scontato_stivaletti = soup_stivaletti.find('span', class_='related__price mr')
prezzo_originale_stivaletti = soup_stivaletti.find('span', class_='related__price mr')
colors_divs  = soup_stivaletti.find_all('div', class_='related__color-name skeleton-effect')
sizes = soup_stivaletti.find('ul', class_='related__select-optgroup-options')

nomi = name_stivaletti.text.strip()
colore =colore_stivaletti.text.strip()
#prezzo_scontato =prezzo_scontato_stivaletti.text.strip()
prezzo_originale = prezzo_originale_stivaletti.text.strip()

nomi = nomi.replace(colore,"")

all_colors = []

# Iterate through all color divs and extract the text
for color_div in colors_divs:
    color_name = color_div.text.strip()
    if color_name:  # Ensure the color name is not empty
        all_colors.append(color_name)

all_size = []

for size in sizes :
    size_val = size.text.strip()
    if size :
        all_size.append(size_val)

# Loop through all colors and sizes to add combinations to the list
for color in all_colors:
    for size in all_size:
        # Add each row as a dictionary
        rows.append({
            'name': nomi,                 # Product name
            'color': color,                  # Color option
            'size': size[:2],                    # Size option
            #'price_sale': prezzo_scontato,  # Sale price
            'price': prezzo_originale[1:]     # Original price
        })

# Create DataFrame from the list of rows
df = pd.DataFrame(rows)

# Display the resulting DataFrame
print(df)


              name              color size  price
0    2790 PLATFORM  AZURE ICE-FAVORIO   35  75.00
1    2790 PLATFORM  AZURE ICE-FAVORIO   35  75.00
2    2790 PLATFORM  AZURE ICE-FAVORIO   36  75.00
3    2790 PLATFORM  AZURE ICE-FAVORIO   37  75.00
4    2790 PLATFORM  AZURE ICE-FAVORIO   37  75.00
..             ...                ...  ...    ...
513  2790 PLATFORM              WHITE   40  75.00
514  2790 PLATFORM              WHITE   41  75.00
515  2790 PLATFORM              WHITE   41  75.00
516  2790 PLATFORM              WHITE   42  75.00
517  2790 PLATFORM              WHITE   42  75.00

[518 rows x 4 columns]
